# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at kafuchino0410@gmail.com



Kaggle: [Link](https://www.kaggle.com/competitions/ntpucsie-ml2022spring-hw5)　Data: [Link](https://drive.google.com/file/d/1u1OjLJLf2kPRJicGSgnBwiWPOtl6eaN6/view?usp=sharing)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [ ]:
!gdown '17kYhMbbnw7r1QvLPqQ6FhrTlaAoZYRXW' --output hw7_data.zip
!unzip -o hw7_data.zip
!nvidia-smi

Downloading...
From: https://drive.google.com/uc?id=17kYhMbbnw7r1QvLPqQ6FhrTlaAoZYRXW
To: /content/hw7_data.zip
100% 11.5M/11.5M [00:00<00:00, 112MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Fri May  5 13:25:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers
!pip install packaging==21.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1


## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset 
from transformers import BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [ ]:
fp16_training = True

if fp16_training:
    !pip install accelerate
    from accelerate import Accelerator
    accelerator = Accelerator(mixed_precision='fp16')
    device = accelerator.device

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.5 MB/s eta 0:00:00


## Load Model and Tokenizer




 

In [ ]:
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
print(tokenizer.tokenize('✦', add_special_tokens=False))
print(tokenizer.tokenize('☺', add_special_tokens=False))
print(tokenizer.tokenize('☆', add_special_tokens=False))

['✦']
['☺']
['☆']


In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

In [ ]:
dev_paragraphs = [i.replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆') for i in dev_paragraphs]
test_paragraphs = [i.replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆') for i in test_paragraphs]

## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [ ]:
print(test_paragraphs_tokenized[test_questions[973]['paragraph_id']].ids)
print(test_paragraphs[test_questions[973]['paragraph_id']])
print(test_questions[973]['question_text'])

[6818, 3309, 8024, 4289, 4415, 2119, 5442, 4634, 4412, 8024, 1762, 3378, 763, 1125, 5471, 7768, 4289, 4415, 2119, 3428, 891, 6166, 8024, 7415, 7768, 4080, 4634, 4638, 4289, 4415, 6121, 4158, 6505, 849, 4696, 4958, 6166, 4638, 1045, 2094, 510, 7442, 2094, 510, 5608, 2094, 5645, 1930, 1046, 511, 6857, 2692, 1456, 5865, 6857, 763, 1825, 4843, 5108, 2094, 3975, 5632, 3176, 1398, 3564, 4638, 3582, 1169, 8024, 1315, 1762, 4696, 4958, 6166, 4638, 2478, 5206, 1125, 5471, 511, 2537, 6857, 3582, 1169, 4496, 4495, 4638, 4289, 4415, 6121, 4158, 3221, 671, 4934, 3963, 4412, 4412, 6496, 511, 3963, 4412, 4294, 2595, 6917, 1377, 5543, 4634, 4495, 1762, 3332, 3160, 4518, 7481, 8024, 3683, 1963, 7079, 7000, 100, 118, 7049, 7000, 100, 4518, 7481, 8024, 969, 6257, 2200, 7079, 7000, 100, 5645, 7049, 7000, 100, 6857, 1060, 1846, 7478, 4828, 5179, 5225, 7768, 6865, 2970, 1762, 671, 6629, 8024, 1179, 3298, 808, 782, 5755, 4197, 6527, 6237, 1765, 1139, 4412, 2206, 7442, 2595, 510, 6631, 2206, 2595, 1350, 7136,

## Dataset and Dataloader

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        self.doc_stride = 300 # Overlapping

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####v
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = int((answer_start_token + answer_end_token) // (2 + random.uniform(-1, 1)))
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
print(test_set[973][0][0])
print(test_set[973][0][1])

tensor([ 101, 1525,  671,  943, 2137, 4415, 2900, 1139, 1392, 4934, 7032, 2253,
        4638, 4229, 2206, 4372, 5645, 7442, 2206, 4372, 4638, 3683, 4372, 6656,
        3984, 2428, 1439, 3633, 3683,  136,  102, 6818, 3309, 8024, 4289, 4415,
        2119, 5442, 4634, 4412, 8024, 1762, 3378,  763, 1125, 5471, 7768, 4289,
        4415, 2119, 3428,  891, 6166, 8024, 7415, 7768, 4080, 4634, 4638, 4289,
        4415, 6121, 4158, 6505,  849, 4696, 4958, 6166, 4638, 1045, 2094,  510,
        7442, 2094,  510, 5608, 2094, 5645, 1930, 1046,  511, 6857, 2692, 1456,
        5865, 6857,  763, 1825, 4843, 5108, 2094, 3975, 5632, 3176, 1398, 3564,
        4638, 3582, 1169, 8024, 1315, 1762, 4696, 4958, 6166, 4638, 2478, 5206,
        1125, 5471,  511, 2537, 6857, 3582, 1169, 4496, 4495, 4638, 4289, 4415,
        6121, 4158, 3221,  671, 4934, 3963, 4412, 4412, 6496,  511, 3963, 4412,
        4294, 2595, 6917, 1377, 5543, 4634, 4495, 1762, 3332, 3160, 4518, 7481,
        8024, 3683, 1963, 7079, 7000,  1

## Function for Evaluation

In [ ]:
def evaluate(data, output, doc_stride, paragraph, question_len):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    doc_start_index = 0
    doc_end_index = 0
    debug_k=0
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            debug_k = k
            doc_start_index = start_index.item() + doc_stride * k - question_len - 2
            doc_end_index = end_index.item() + doc_stride * k - question_len - 2
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    if '[UNK]' in answer:
        print('Change ans: ', answer)
        old_ans = answer
        answer = paragraph[doc_start_index:doc_end_index+1]
        print('To new ans: ', answer)
        if old_ans[0] != answer[0]:
          print(paragraph)
          print(data[0][0][debug_k])

    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace(' ','')

## Training

In [ ]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps= 0, num_training_steps=len(train_loader) * num_epoch, num_cycles = num_epoch)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        step += 1
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                ans = evaluate(data, output, 300,dev_paragraphs[dev_questions[i]['paragraph_id']], len(dev_questions[i]['question_text']))
                dev_acc +=  ans == dev_questions[i]["answer_text"]
                if (ans != dev_questions[i]["answer_text"]):
                  print('Bert ans:', ans)
                  print('Real ans:', dev_questions[i]["answer_text"])
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3242 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 2.717, acc = 0.297
Epoch 1 | Step 200 | loss = 1.004, acc = 0.642
Epoch 1 | Step 300 | loss = 0.804, acc = 0.674
Epoch 1 | Step 400 | loss = 0.800, acc = 0.681
Epoch 1 | Step 500 | loss = 0.651, acc = 0.732
Epoch 1 | Step 600 | loss = 0.804, acc = 0.711
Epoch 1 | Step 700 | loss = 0.625, acc = 0.757
Epoch 1 | Step 800 | loss = 0.596, acc = 0.727
Epoch 1 | Step 900 | loss = 0.614, acc = 0.752
Epoch 1 | Step 1000 | loss = 0.606, acc = 0.746
Epoch 1 | Step 1100 | loss = 0.595, acc = 0.767
Epoch 1 | Step 1200 | loss = 0.555, acc = 0.770
Epoch 1 | Step 1300 | loss = 0.473, acc = 0.801
Epoch 1 | Step 1400 | loss = 0.494, acc = 0.792
Epoch 1 | Step 1500 | loss = 0.543, acc = 0.760
Epoch 1 | Step 1600 | loss = 0.508, acc = 0.784
Epoch 1 | Step 1700 | loss = 0.499, acc = 0.779
Epoch 1 | Step 1800 | loss = 0.420, acc = 0.805
Epoch 1 | Step 1900 | loss = 0.516, acc = 0.780
Epoch 1 | Step 2000 | loss = 0.493, acc = 0.790
Epoch 1 | Step 2100 | loss = 0.524, acc = 0.785
E

  0%|          | 0/3524 [00:00<?, ?it/s]

Bert ans: 未來9日
Real ans: 9日
Bert ans: 紅土
Real ans: 法網
Bert ans: 鑑於新教徒對聖經正典的質疑
Real ans: 新教徒對聖經正典的質疑
Bert ans: 新加坡在近十五人均gdp都比香港
Real ans: 新加坡
Bert ans: 蝦夷地
Real ans: 「蝦夷地」
Bert ans: 公元前一萬年
Real ans: 公元前一萬年前
Bert ans: 傳統上會將消費者視為是被動的
Real ans: 消費者
Bert ans: 《德意志民主共和國憲法》第1章，第1條
Real ans: 1949年
Bert ans: 英制貴族系統
Real ans: 世襲
Bert ans: 降雨的不均衡和英國人的經濟管理政策失誤
Real ans: 降雨的不均衡
Bert ans: 闢為公園
Real ans: 闢為公園，正式對民眾開放
Bert ans: 《周禮·考工記》
Real ans: 周禮·考工記
Bert ans: 商朝時期
Real ans: 商朝
Bert ans: 四面環海
Real ans: 台灣四面環海
Bert ans: 法西斯獨裁政權
Real ans: 法西斯獨裁
Bert ans: 蘇維埃俄國
Real ans: 蘇維埃俄國政府
Bert ans: 近衛頭銜
Real ans: 近衛稱號
Bert ans: 《國際動物命名規約》
Real ans: 國際動物命名規約
Bert ans: 敘利亞
Real ans: 烏茲別克
Bert ans: 亞塞拜然人村落
Real ans: 亞塞拜然人
Bert ans: 香港之女
Real ans: 「香港之女」
Bert ans: 
Real ans: 岩石圈
Bert ans: 教師教學質量不好
Real ans: 教師教學質量不好，學生要求教學改革
Bert ans: 芬蘭
Real ans: 芬蘭民族
Bert ans: 2007年
Real ans: 2007年5月
Bert ans: 比喻苦難
Real ans: 苦難
Bert ans: 恢復了洪武年間廢除的錦衣衛
Real ans: 對大臣有所猜忌
Bert ans: 
Real ans: 政客
Bert ans: 蘇聯在四強先擊敗捷克斯洛伐克，及後在決賽以2比1戰勝南

...

Bert ans: 未來9日
Real ans: 9日
Bert ans: 紅土
Real ans: 法網
Bert ans: 鑑於新教徒對聖經正典的質疑
Real ans: 新教徒對聖經正典的質疑
Bert ans: 蝦夷地
Real ans: 「蝦夷地」
Bert ans: 消費者視為是被動的
Real ans: 消費者
Bert ans: 《德意志民主共和國憲法
Real ans: 1949年
Bert ans: 英制
Real ans: 世襲
Bert ans: 降雨的不均衡和英國人的經濟管理政策失誤
Real ans: 降雨的不均衡
Bert ans: 闢為公園
Real ans: 闢為公園，正式對民眾開放
Bert ans: 《周禮·考工記
Real ans: 周禮·考工記
Bert ans: 商朝時期
Real ans: 商朝
Bert ans: 四面環海
Real ans: 台灣四面環海
Bert ans: 法西斯
Real ans: 法西斯獨裁
Bert ans: 蘇維埃俄國
Real ans: 蘇維埃俄國政府
Bert ans: 劉德華
Real ans: 梅艷芳
Bert ans: 近衛頭銜
Real ans: 近衛稱號
Bert ans: 《國際動物命名規約》
Real ans: 國際動物命名規約
Bert ans: 敘利亞
Real ans: 烏茲別克
Bert ans: 亞塞拜然人村落
Real ans: 亞塞拜然人
Bert ans: 香港之女
Real ans: 「香港之女」
Bert ans: 
Real ans: 岩石圈
Bert ans: 嘉陵道
Real ans: 嘉陵道籍
Bert ans: 廈門大學教師教學質量不好
Real ans: 教師教學質量不好，學生要求教學改革
Bert ans: 芬蘭
Real ans: 芬蘭民族
Bert ans: 比喻苦難
Real ans: 苦難
Bert ans: 恢復了洪武年間廢除的錦衣衛
Real ans: 對大臣有所猜忌
Bert ans: 
Real ans: 政客
Bert ans: 一年
Real ans: 一年左右
Bert ans: 三千
Real ans: 不到三千
Bert ans: 42.00
Real ans: 42
Bert ans: 過早引爆
Real ans

  0%|          | 0/3242 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.254, acc = 0.874
Epoch 3 | Step 200 | loss = 0.249, acc = 0.881
Epoch 3 | Step 300 | loss = 0.315, acc = 0.852
Epoch 3 | Step 400 | loss = 0.331, acc = 0.862
Epoch 3 | Step 500 | loss = 0.319, acc = 0.852
Epoch 3 | Step 600 | loss = 0.280, acc = 0.869
Epoch 3 | Step 700 | loss = 0.280, acc = 0.864
Epoch 3 | Step 800 | loss = 0.284, acc = 0.862
Epoch 3 | Step 900 | loss = 0.261, acc = 0.881
Epoch 3 | Step 1000 | loss = 0.288, acc = 0.854
Epoch 3 | Step 1100 | loss = 0.275, acc = 0.864
Epoch 3 | Step 1200 | loss = 0.257, acc = 0.866
Epoch 3 | Step 1300 | loss = 0.265, acc = 0.885
Epoch 3 | Step 1400 | loss = 0.265, acc = 0.880
Epoch 3 | Step 1500 | loss = 0.277, acc = 0.873
Epoch 3 | Step 1600 | loss = 0.265, acc = 0.874
Epoch 3 | Step 1700 | loss = 0.285, acc = 0.869
Epoch 3 | Step 1800 | loss = 0.268, acc = 0.874
Epoch 3 | Step 1900 | loss = 0.228, acc = 0.895
Epoch 3 | Step 2000 | loss = 0.303, acc = 0.877
Epoch 3 | Step 2100 | loss = 0.282, acc = 0.874
E

  0%|          | 0/3524 [00:00<?, ?it/s]

Bert ans: 1億7千5百萬美元
Real ans: 3500萬美元
Bert ans: 未來9日
Real ans: 9日
Bert ans: 紅土
Real ans: 法網
Bert ans: 鑑於新教徒對聖經正典的質疑
Real ans: 新教徒對聖經正典的質疑
Bert ans: 蝦夷地
Real ans: 「蝦夷地」
Bert ans: 公元前一萬年
Real ans: 公元前一萬年前
Bert ans: 《德意志民主共和國憲法》
Real ans: 1949年
Bert ans: 英制
Real ans: 世襲
Bert ans: 闢為公園
Real ans: 闢為公園，正式對民眾開放
Bert ans: 《周禮·考工記
Real ans: 周禮·考工記
Bert ans: 日軍開始拆除上海南站及其附近長約1.5公里鐵路。同時修復上海北站
Real ans: 日軍
Bert ans: 商朝時期
Real ans: 商朝
Bert ans: 四面環海
Real ans: 台灣四面環海
Bert ans: 法西斯
Real ans: 法西斯獨裁
Bert ans: 蘇維埃俄國
Real ans: 蘇維埃俄國政府
Bert ans: 近衛頭銜
Real ans: 近衛稱號
Bert ans: 《國際動物命名規約》
Real ans: 國際動物命名規約
Bert ans: 敘利亞
Real ans: 烏茲別克
Bert ans: 亞塞拜然人村落
Real ans: 亞塞拜然人
Bert ans: 香港之女
Real ans: 「香港之女」
Bert ans: 對閩南地形較為熟識，在江東橋附近安排了許多伏兵
Real ans: 對閩南地形較為熟識
Bert ans: 芬蘭
Real ans: 芬蘭民族
Bert ans: 比喻苦難
Real ans: 苦難
Bert ans: 朱棣是造反稱帝的，對大臣有所猜忌
Real ans: 對大臣有所猜忌
Bert ans: 南斯拉夫
Real ans: 蘇聯
Bert ans: 一年
Real ans: 一年左右
Bert ans: 三千
Real ans: 不到三千
Bert ans: 42.00
Real ans: 42
Bert ans: 過早引爆
Real ans: 促使彈頭過早引爆
Bert ans: 19

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, 300, test_paragraphs[test_questions[i]['paragraph_id']], len(test_questions[i]['question_text'])))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/1000 [00:00<?, ?it/s]

Change ans:  蕭 [UNK]
To new ans:  ，大
820年唐憲宗在大明宮被宦官毒死，河北三鎮復叛，中興時期結束。821年盧龍劉總離職，唐室派張弘靖接管。張弘靖管理不當，盧龍兵擁護朱克融叛變。移鎮成德的田弘正被將領王庭湊奪位殺害。魏博田布被軍隊迫死，魏博軍擁護史憲誠叛變，至此河北三鎮復叛。當河北未叛之時，大臣蕭俛、段文昌建議國家裁軍。如今被裁之兵都投奔河北三鎮，助長其勢。然而此後的河北三鎮並非持續強盛，唐敬宗與唐武宗期間，河北三鎮大多受制其強兵，有時還被部下篡位，遠遠不如當初的跋扈。而各地藩鎮依然聽命於中央，直到黃巢之亂為止。唐朝中央的政治大權大多由皇帝與宰相掌控，但在天寶之後轉變成皇帝與內廷宦官的聯合，外廷宰相變成政治上的二流角色。
tensor([  101,  2832,  1944,  3777,  1266,   676,  7120,  4638,  1070,  1333,
         3315,  3221,  1728,  4158,  6306,  4638,  2456,  6359,  5445,  6158,
         6161,  4638,  8043,   102, 10398,  2399,  1538,  2740,  2134,  1762,
         1920,  3209,  2152,  6158,  2149,  2135,  3681,  3647,  8024,  3777,
         1266,   676,  7120,  2541,  1361,  8024,   704,  5646,  3229,  3309,
         5178,  3338,   511,  8460,  8148,  2399,  4678,  7983,  1208,  5244,
         7431,  5480,  8024,  1538,  2147,  3836,  2484,  2473,  7473,  2970,
         5052,   511,  2484,  2473,  7473,  5052,  4415,   679,  4534,  8024,
         4678,  7983,  1070,  3075,  6362, 

In [ ]:
from google.colab import files
!zip -r 'result.zip' 'result.csv' -9
!zip -r 'saved_model.zip' 'saved_model' -9
files.download('result.zip')
files.download('saved_model.zip')

  adding: result.csv (deflated 39%)
  adding: saved_model/ (stored 0%)
  adding: saved_model/config.json (deflated 51%)
  adding: saved_model/pytorch_model.bin (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>